In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# construct the file path

# name of raw data file
fname_raw = "water_dept_weather_raw.csv"

# relative path of raw file
fpath_raw = "./data/" + fname_raw

# import csv file as pandas dataframe
df = pd.read_csv(fpath_raw)

df.head()

,STATION,NAME,DATE,DAPR,MDPR,MDSF,PRCP,SNOW,SNWD,TMAX,TMIN,TOBS,WT01,WT03,WT04,WT05,WT06,WT11
0,USC00052223,"DENVER WATER DEPARTMENT, CO US",2005-01-01,NaN,NaN,NaN,0.00,0.0,0.0,28.0,20.0,20.0,NaN,NaN,NaN,NaN,NaN,NaN
1,USC00052223,"DENVER WATER DEPARTMENT, CO US",2005-01-02,NaN,NaN,NaN,0.00,0.0,0.0,45.0,20.0,28.0,NaN,NaN,NaN,NaN,NaN,NaN
2,USC00052223,"DENVER WATER DEPARTMENT, CO US",2005-01-03,NaN,NaN,NaN,0.00,0.0,0.0,38.0,18.0,23.0,NaN,NaN,NaN,NaN,NaN,NaN
3,USC00052223,"DENVER WATER DEPARTMENT, CO US",2005-01-04,NaN,NaN,NaN,0.00,0.0,0.0,43.0,19.0,28.0,NaN,NaN,NaN,NaN,NaN,NaN
4,USC00052223,"DENVER WATER DEPARTMENT, CO US",2005-01-05,NaN,NaN,NaN,0.02,1.7,2.0,28.0,3.0,7.0,NaN,NaN,NaN,NaN,NaN,NaN


### The index is currently the standard RangeIndex. We want to make it such that the date acts as the index for each data point.

In [3]:
# change the index of the dataframe to be DATE column
df.set_index("DATE", inplace=True)

# turn the index from string into a datetime object
df.index = pd.to_datetime(df.index)

df.head()

,STATION,NAME,DAPR,MDPR,MDSF,PRCP,SNOW,SNWD,TMAX,TMIN,TOBS,WT01,WT03,WT04,WT05,WT06,WT11
DATE,,,,,,,,,,,,,,,,,
2005-01-01,USC00052223,"DENVER WATER DEPARTMENT, CO US",NaN,NaN,NaN,0.00,0.0,0.0,28.0,20.0,20.0,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-02,USC00052223,"DENVER WATER DEPARTMENT, CO US",NaN,NaN,NaN,0.00,0.0,0.0,45.0,20.0,28.0,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-03,USC00052223,"DENVER WATER DEPARTMENT, CO US",NaN,NaN,NaN,0.00,0.0,0.0,38.0,18.0,23.0,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-04,USC00052223,"DENVER WATER DEPARTMENT, CO US",NaN,NaN,NaN,0.00,0.0,0.0,43.0,19.0,28.0,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-05,USC00052223,"DENVER WATER DEPARTMENT, CO US",NaN,NaN,NaN,0.02,1.7,2.0,28.0,3.0,7.0,NaN,NaN,NaN,NaN,NaN,NaN


### We see that there are a lot of NaN values in the dataset. Let's see how many missing values there are in comparison to the number of observations.

In [4]:
# number of observations
total_observations = df.shape[0]
print(f'Number of observations: {total_observations}')

# number of NaN values for each column
nan_values = pd.Series(df.apply(pd.isnull).sum(),
                       name="NaN count")

print(f'NaN values in each column:')

# sort rows by number of
nan_values.sort_values()

Number of observations: 7288
NaN values in each column:


,NaN count
STATION,0
NAME,0
SNOW,61
SNWD,91
PRCP,109
TMAX,121
TMIN,128
TOBS,1061
WT03,7154
MDPR,7257


### There is a significant jump in NaN values between the TMIN and TOBS columns. Therefore, we will tolerate all columns containing NaNs values less than or equal to the WSF5 column. First we create a list of all the columns we will accept.

In [5]:
# all acceptable columns and their corresponding Nan value counts
acceptable_columns = nan_values[nan_values <= nan_values["TMIN"]]

# turn acceptable_columns into a list of the columns with low Nan counts
acceptable_columns = acceptable_columns.index.to_list()

acceptable_columns

['STATION', 'NAME', 'PRCP', 'SNOW', 'SNWD', 'TMAX', 'TMIN']

### Now that we know which columns are acceptable, we can remove all other columns.

In [6]:
# remove columns with too many NaN values
df = df[acceptable_columns].copy()

df

,STATION,NAME,PRCP,SNOW,SNWD,TMAX,TMIN
DATE,,,,,,,
2005-01-01,USC00052223,"DENVER WATER DEPARTMENT, CO US",0.00,0.0,0.0,28.0,20.0
2005-01-02,USC00052223,"DENVER WATER DEPARTMENT, CO US",0.00,0.0,0.0,45.0,20.0
2005-01-03,USC00052223,"DENVER WATER DEPARTMENT, CO US",0.00,0.0,0.0,38.0,18.0
2005-01-04,USC00052223,"DENVER WATER DEPARTMENT, CO US",0.00,0.0,0.0,43.0,19.0
2005-01-05,USC00052223,"DENVER WATER DEPARTMENT, CO US",0.02,1.7,2.0,28.0,3.0
...,...,...,...,...,...,...,...
2025-03-25,USC00052223,"DENVER WATER DEPARTMENT, CO US",0.00,0.0,0.0,68.0,46.0
2025-03-26,USC00052223,"DENVER WATER DEPARTMENT, CO US",0.00,0.0,0.0,72.0,41.0
2025-03-27,USC00052223,"DENVER WATER DEPARTMENT, CO US",0.00,0.0,0.0,77.0,41.0


### Let's check to make sure that no row is completely filled with NaN

In [7]:
# all rows whose values are exclusively NaN (aside from STATION and NAME)
nan_rows = df[df.iloc[:,2:].isnull().all(axis=1)]

nan_rows

,STATION,NAME,PRCP,SNOW,SNWD,TMAX,TMIN
DATE,,,,,,,


### Next we will fill in the remaining NaN values. In this context, it makes sense since the weather typically doesn't change dramatically between days. We will fill NaN values using the most recent previous value.

In [8]:
# fill missing values using previous value
df = df.ffill()

# number of null values for each column
nan_values_ffill = pd.Series(df.apply(pd.isnull).sum(),
                       name="NaN count")
print(f'Null values in each column after ffill():')
nan_values_ffill

Null values in each column after ffill():


,NaN count
STATION,0
NAME,0
PRCP,0
SNOW,0
SNWD,0
TMAX,0
TMIN,0


### This is what the final dataset looks like:

In [9]:
df

,STATION,NAME,PRCP,SNOW,SNWD,TMAX,TMIN
DATE,,,,,,,
2005-01-01,USC00052223,"DENVER WATER DEPARTMENT, CO US",0.00,0.0,0.0,28.0,20.0
2005-01-02,USC00052223,"DENVER WATER DEPARTMENT, CO US",0.00,0.0,0.0,45.0,20.0
2005-01-03,USC00052223,"DENVER WATER DEPARTMENT, CO US",0.00,0.0,0.0,38.0,18.0
2005-01-04,USC00052223,"DENVER WATER DEPARTMENT, CO US",0.00,0.0,0.0,43.0,19.0
2005-01-05,USC00052223,"DENVER WATER DEPARTMENT, CO US",0.02,1.7,2.0,28.0,3.0
...,...,...,...,...,...,...,...
2025-03-25,USC00052223,"DENVER WATER DEPARTMENT, CO US",0.00,0.0,0.0,68.0,46.0
2025-03-26,USC00052223,"DENVER WATER DEPARTMENT, CO US",0.00,0.0,0.0,72.0,41.0
2025-03-27,USC00052223,"DENVER WATER DEPARTMENT, CO US",0.00,0.0,0.0,77.0,41.0


### Now we can export the dataframe as a csv file.

In [10]:
# cleaned file name
fname_clean = fname_raw[:-7] + "clean.csv" # change 'raw' to 'clean

fpath_clean = "./data/" + fname_clean

df.to_csv(fpath_clean, index=True)